In [1]:
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()
from matplotlib import pyplot as plt, cm, colors
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D

sns.set_style("ticks")
sns.set_style("ticks")
sns.despine()
from matplotlib import pyplot as plt, cm, colors
from tqdm.notebook import tqdm

import glob
import os.path as ospath
import os
from sys import executable
from subprocess import check_output
from PyQt5.QtWidgets import QFileDialog, QApplication
from IPython.display import HTML

from scipy import optimize
from scipy.spatial import distance
from scipy import linalg
from scipy import signal
from sklearn.cluster import MeanShift, estimate_bandwidth

from picasso import io
from picasso.postprocess import link, compute_dark_times
from picasso.render import render
from picasso.gui.render import estimate_kinetic_rate, fit_cum_exp

# define visualization options 
%matplotlib inline
%gui qt

# define colors:
blue = "#4C72B0"
orange = "#DD8452"
red = "#C44E52"
gray = "#90A8CE"


def OpenFileDialog():
    file = check_output([executable, __file__])
    return file.strip()


def gui_fname(dir=None):
    """
    Select a file via a dialog and return the file name.
    """
    if dir is None: 
        dir ="./"

    app = QApplication([dir])
    fname = QFileDialog.getExistingDirectory(None, "Select a folder...", 
            dir)
    if isinstance(fname, tuple):
        return fname[0]
    else: 
        return str(fname)


def load_files(dirname):
    
    os.chdir(dirname)
    files = glob.glob("*.hdf5")
    
    if files:
        print("{} HDF5 files found.".format(len(files)))
    else:
        print("No HDF5 files found at: {}".format(dirname))
            
    return files


def load_data(path):

    try:
        locs, info = io.load_locs(path)
    except io.NoMetadataFileError:
        return None, None, None
    
    try:
        pixelsize = info[1]["Pixelsize"]
    except:  
        print("No pixelsize found in yaml file. Default 130 nm used.")
  
    if hasattr(locs, "x_pick_rot"):

        # convert px to nm
        locs.x *= pixelsize
        locs.y *= pixelsize
        locs.x_pick_rot *= pixelsize
        locs.y_pick_rot *= pixelsize
        
        return locs, info, pixelsize
    else:
        return None, None, None


def double_gaus(x,a,x0,sigma, b, x1, sigma1):
    return a*np.exp(-(x-x0)**2/(2*sigma**2)) + b*np.exp(-(x-x1)**2/(2*sigma1**2))

def gaus(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2)) 


def find_peaks(data, binning=100, axes="y"):
    
    if axes == "y":
        column = "y_pick_rot"
    elif axes == "x":
        column = "x_pick_rot"
    elif axes == "xyz":
        column = 2
    
    # find peaks
    bandwidth = estimate_bandwidth(data[column].reshape(-1, 1), quantile=0.2, n_samples=binning)
    #print("estimated bandwidth: "+str(bandwidth))
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    ms.fit(data[column].reshape(-1, 1))
    labels = ms.labels_
    peaks = np.sort(ms.cluster_centers_[0:2],axis=None) # assuming that there are two large peaks from the two rings
    peak1 = np.float(peaks[0])
    peak2 = np.float(peaks[1])
    estimated_peaks = {0:peak1, 1:peak2}

    # histogram
    n, bins = np.histogram(data[column], bins=binning)
    bins = bins[1:]
    hist_data = [n, bins]

    # fit peaks
    p0 = [peak1/2, peak1, 40, peak2/2, peak2, 40]
    try:
        p_fit, p_cov = optimize.curve_fit(double_gaus, bins, n, p0=p0)
    except:
        p_fit = [0,0,0,1000,1000,1000]
        
    
    # check order of fitted peaks (peak 1 < peak 2)
    if p_fit[1] > p_fit[4]:
        p_temp = p_fit.copy()
        p_fit[0:3]=p_temp[3:6]
        p_fit[3:6]=p_temp[0:3]
       
        
    # p_fit: amplitued_1, center_1, width_1, amplitude_2, center_2, width_2
        
    return estimated_peaks, p_fit, hist_data


def plot_peak_dist(data, hist_data, peaks, p_fit, binning=100,cutoff=1, axes="y", ax=None):
    
    if ax is None:
        ax = plt.gca()
        
    if axes == "y":
        column = "y_pick_rot"
    elif axes == "x":
        column = "x_pick_rot"
    elif axes == "xyz":
        column == 1
        
    peak1 = peaks[0]
    peak2 = peaks[1]
    
    n = hist_data[0]
    bins = hist_data[1]
    
    #fig, ax = plt.subplots(figsize=(9, 5))
    binwidth = (max(bins)-min(bins)) / binning
    ax.bar(bins, n, width=binwidth, color=gray)
    xlin = np.linspace(0, data[column].max(), 1000)
    ax.plot(xlin, gaus(xlin,*p_fit[0:3]), c=red, linewidth=2)
    ax.plot(xlin, gaus(xlin,*p_fit[3:6]), c=red, linewidth=2)
    ax.axvline(p_fit[1]-(p_fit[2]*cutoff),c=blue, linewidth=2, linestyle="--")
    ax.axvline(p_fit[1]+(p_fit[2]*cutoff),c=blue, linewidth=2, linestyle="--")
    ax.axvline(p_fit[4]-(p_fit[5]*cutoff),c=orange, linewidth=2, linestyle="--")
    ax.axvline(p_fit[4]+(p_fit[5]*cutoff),c=orange, linewidth=2, linestyle="--")
    ax.set_title("Line profile",loc="left",fontsize=14)
    ax.set_xlabel("y (nm)")
    ax.set_ylabel("Counts")  
    ax.text(0.15,
            0.7,
            ("Estimated Peaks:\n"
            "Peak 1 at {:.1f} nm\n"
            "Peak 2 at {:.1f} nm\n"
            "Fitted Peaks:\n"
            "Peak 1 at {:.1f} nm\n"
            "Peak 2 at {:.1f} nm").format(peak1,peak2,p_fit[1],p_fit[4]),
            horizontalalignment="center",
            verticalalignment="center",
            transform = ax.transAxes,
            fontsize=12)

    return ax


def isolate_ring(data, c, w, axes="y", cutoff=1.0):
    
    if axes == "y":
        column = "y_pick_rot"
    elif axes == "x":
        column = "x_pick_rot"
        
    ring_data = data[np.where((data[column]>(c-(cutoff*w))) & (data[column] <(c+(cutoff*w))))]
    return ring_data


def rodrigues_rot(P, n0, n1):
    # adapted from https://meshlogic.github.io/posts/jupyter/curve-fitting/fitting-a-circle-to-cluster-of-3d-points/
    # If P is only 1d array (coords of single point), fix it to be matrix
    if P.ndim == 1:
        P = P[np.newaxis,:]
    
    # Get vector of rotation k and angle theta
    n0 = n0/linalg.norm(n0)
    n1 = n1/linalg.norm(n1)
    k = np.cross(n0,n1)
    k = k/linalg.norm(k)
    theta =np.arccos(np.dot(n0,n1))
    
    # Compute rotated points
    P_rot = np.zeros((len(P),3))
    for i in range(len(P)):
        P_rot[i] = P[i]*np.cos(theta) + np.cross(k,P[i])*np.sin(theta) + k*np.dot(k,P[i])*(1-np.cos(theta))

    return P_rot


def rotate_ring(XYZ): 
    # Fitting plane by SVD for the mean-centered data
    # Eq. of plane is <p,n> + d = 0, where p is a point on plane and n is normal vector
       
    # Normal vector of fitting plane is given by 3rd column in V
    # Note linalg.svd returns V^T, so we need to select 3rd row from V^T
    ring_mean = XYZ.mean(axis=0)
    ring_centered = XYZ - ring_mean
    U,s,V = linalg.svd(ring_centered)
    normal = V[2,:]
    d = -np.dot(ring_mean, normal) 
        
    n0 = normal # new z axes
    n1 = [0,0,1] # old z axes

    ring_rot = rodrigues_rot(ring_centered, n0, n1)
    
    return ring_rot, normal


def plot_3d_ring(ring_rot,dist, ax=None):
    
    ax.scatter(ring_rot[0][:,0], ring_rot[0][:,1], ring_rot[0][:,2]-dist/2,c=blue,label="Spore", alpha=0.5)
    ax.scatter(ring_rot[1][:,0], ring_rot[1][:,1], ring_rot[1][:,2]+dist/2,c=orange,label="Mother", alpha=0.5)
    ax.set_xlabel("x (nm)")
    ax.set_ylabel("y (nm)")
    ax.set_zlabel("z (nm)")
    ax.legend(loc="best",labelspacing=0.1)
    #ax.view_init(elev=2., azim=10.)
    ax.view_init(elev=30., azim=10.)
    set_axes_equal_3d(ax)

    return ax
    
    
def set_axes_equal_3d(ax):
    limits = np.array([ax.get_xlim3d(), ax.get_ylim3d(), ax.get_zlim3d()])
    spans = abs(limits[:,0] - limits[:,1])
    centers = np.mean(limits, axis=1)
    radius = 0.5 * max(spans)
    ax.set_xlim3d([centers[0]-radius, centers[0]+radius])
    ax.set_ylim3d([centers[1]-radius, centers[1]+radius])
    ax.set_zlim3d([centers[2]-radius, centers[2]+radius])
    

def angle_between(u, v, n=None):
    if n is None:
        return np.arctan2(np.linalg.norm(np.cross(u,v)), np.dot(u,v))*180/np.pi
    else:
        return np.arctan2(np.dot(n,np.cross(u,v)), np.dot(u,v))*180/np.pi


def calc_R(x, y, xc, yc):
    # adapted from https://gist.github.com/lorenzoriano/6799568
    """ calculate the distance of each 2D points from the center (xc, yc) """
    return np.sqrt((x-xc)**2 + (y-yc)**2)


def f(c, x, y):
    """ calculate the algebraic distance between the data points and the mean circle centered at c=(xc, yc) """
    Ri = calc_R(x, y, *c)
    return Ri - np.median(Ri)


def leastsq_circle(x,y):
    
    x_m = np.median(x)
    y_m = np.median(y)
    center_estimate = x_m, y_m
    center, ier = optimize.leastsq(f, center_estimate, args=(x,y))
    xc, yc = center
    Ri       = calc_R(x, y, *center)
    R        = np.median(Ri)
    residu   = np.sum((Ri - R)**2)
    return xc, yc, R, residu


def plot_data_circle(x, y, xc, yc, R, id, center=True, ax=None):
    
    if id == 0:
        label = "spore"
        color = blue
    else:
        label = "mother"
        color = orange
    
    if ax is None:
        ax = plt.gca()
    
    if center:
        x -= xc
        y -= yc
        xc = 0
        yc = 0 
        
    #f, ax = plt.subplots(figsize=(5,5))  #figsize=(7, 5.4), dpi=72,
    theta_fit = np.linspace(-np.pi,np.pi, 180)
    x_fit = xc + R*np.cos(theta_fit)
    y_fit = yc + R*np.sin(theta_fit)
    
    # plot fit
    ax.plot(x_fit, y_fit, label="Fitted circle", lw=2, c=red)
    ax.plot([xc], [yc], mec="y", mew=1,  c=red)
    
    # plot data
    ax.scatter(x, y, alpha=0.4,  label="Projected locs", marker=".", c=color)
    
    ax.set_xlabel("x rotated (nm)")
    ax.set_ylabel("y rotated (nm)")
    #ax.axis("equal")
    ax.set_xlim([-800,800])
    ax.set_aspect("equal",adjustable="datalim")
    
    ax.legend(loc="best", labelspacing=0.1)
    ax.set_title("Least squares circle {}\n"
                 "Fit radius: {:.1f} nm".format(label,R),loc="left",fontsize=14)
    
    return 


def check_consistency(ring_par):
    
    ev = 0
    
    amp_1, c_1, w_1 = ring_par[0]
    amp_2, c_2, w_2 = ring_par[1]
    
    if amp_1/amp_2 >= 0.3 and amp_2/amp_1 >= 0.3:
        ev +=1 
        
    if abs(c_1-c_2) > 30 and abs(c_1-c_2) < 300:
        ev +=1
        
    if w_1/w_2 >= 0.3 and w_2/w_1 >=0.3:
        ev +=1
        
    return (ev==3)

def plot_cum_exp(pooled_locs, fit_result_len, fit_result_dark, id, ax=None):
    
    if id == 0:
        color = blue
        label = 'spore'
    if id == 1:
        color = orange
        label = 'mother'

    if ax is None:
        ax = plt.gca()
    
    data = pooled_locs.dark
    data.sort()
    y = np.arange(1, len(data) + 1)
       
    a = fit_result_dark.best_values["a"]
    t = fit_result_dark.best_values["t"]
    c = fit_result_dark.best_values["c"]

    ax.set_title(
        "Dark time (cumulative) {}\n"
        r"$Fit: {:.2f}\cdot(1-exp(x/{:.2f}))+{:.2f}$".format(label, a, t, c),loc="left",fontsize=14)
    data = pooled_locs.dark
    data.sort()
    y = np.arange(1, len(data) + 1)

    ax.semilogx(data, y, c=color, label="Data")
    ax.semilogx(data, fit_result_dark.best_fit, c=red, label="Fit")
    ax.legend(loc="best")
    ax.set_xlabel("Duration (frames)")
    ax.set_ylabel("Frequency")
    
    return ax


def save_ring_locs(locs, info, path, file_id, pick, id, link=False):
    
    if link:
        ending = "_link.hdf5"
    else:
        ending = ".hdf5"
       
    locs.x /= 130
    locs.y /= 130
    
    locs_name = "file_{}_pick_{}_ring_{}{}".format(file_id, pick, id, ending)
    locs_path = os.path.join(path,"ring_locs")
    locs_path_name = os.path.join(locs_path, locs_name)
    
    if not os.path.isdir(locs_path):
        os.makedirs(locs_path)
    
    io.save_locs(locs_path_name, locs, info)

    
def export_pick_img(locs, path, file_id, pick, id, link=False):
    
    if link:
        ending = "_link.png"
    else:
        ending = ".png"
    
    pixelsize = 130

    export_locs = locs.copy()
    
    export_locs.x /= pixelsize
    export_locs.y /= pixelsize
    
    x_min = np.min(export_locs.x)    
    x_max = np.max(export_locs.x)
    y_min = np.min(export_locs.y)
    y_max = np.max(export_locs.y)

    viewport =  (y_min, x_min), (y_max, x_max)
    oversampling = 50
    len_x, image = render(export_locs, viewport = viewport, oversampling=oversampling, blur_method="smooth")
    
    img_name = "file_{}_pick_{}_ring_{}{}".format(file_id, pick, id, ending)
    img_path = os.path.join(path,"ring_images")
    img_path_name = os.path.join(img_path,img_name)
    
    if not os.path.isdir(img_path):
        os.makedirs(img_path)
    
    plt.imsave(img_path_name, image, cmap="hot")

## Load data

In [4]:
path = gui_fname()
filenames = load_files(path)

plotting = True
max_dist = 130 #nanometer
max_dark_time = 15 #frames
binning = 50 # binning for peak histogram


28 HDF5 files found.


## Main loop

In [13]:
# initialize containers
ring_data = []
ring_locs = {}
ring_locs_linked = {}
ring_kinetics = {}
ring_kinetics_fit = {}
ring_radii = {}
ring_rot = {}
ring_angles = {}
center = {}

rings_excluded = []
ring_n_events = {}
circle_plots = {}
file_id = 0

analysis_folder = os.path.join(path, "analysis")
analysis_folder_excluded = os.path.join(analysis_folder, "excluded")

# image export settings
img_format = ".svg"
dpi = 100

# cutoff is mutliplied to the sigma of the peak fit for the seperation of the two rings
# smaller cutoff means smaller ring sections
cutoff = 1.25

#prepare analysis folder
if not os.path.isdir(analysis_folder):
    os.makedirs(analysis_folder)
    os.makedirs(analysis_folder_excluded)

#iterate over locs in directory:
for filename in tqdm(filenames, desc="Processing files"):
    
    file_id +=1
    
    #load locs and convert distances from px to nm (Attention!)
    locs, info, pixelsize = load_data(os.path.join(path,filename))
    
    if locs is None:
        print("File {} not loaded.".format(filename))
        continue
    
    # iterate over picks in a file
    for pick in tqdm(np.unique(locs.group), desc="Processing picks"):
     
        # select locs from pick
        pick_locs = locs[locs.group == pick]

        ################
        # (1) estimate the postition of the two rings using a histogram along the pick direction
        #  a ring should yield a gaussian peak (2D projection in XY)
        ################
        
        estimated_peaks, r_par, hist_data = find_peaks(pick_locs, binning=binning, axes="y")
        # estimated peaks form MeanShift analysis to initialize guassian fits
        # r_par yields an array containing: [amplitued_1, center_1, width_1, amplitude_2, center_2, width_2]

        # seperate ring parameter to the corresponding ring
        ring_parameter = []
        for i in range(0, 4, 3):
            ring_parameter.append(r_par[i: i+3])
        #print("Correct order: {}".format(str(ring_parameter[0][1]<ring_parameter[1][1])))
            
        if check_consistency(ring_parameter):
        # check three criteria:
        # i   distance between the peaks (30 nm < distance < 300 nm)
        # ii  width of the gaussian fits of the peaks (ratio needs to be larger then 0.2 in both cases)
        # iii "signal strength" of both rings should be similar (amplitude of the gaussian peak should be larger then 0.3)
            
            # set up plot with gridspec
            fig = plt.figure(figsize=(14, 16), constrained_layout=True)
            gs = fig.add_gridspec(3, 2)
            fig.suptitle(("Pick {} - Ring analysis\n"
                          "File: {}").format(pick,filename),
                          fontsize=16,
                          ha="center")
            
            ax1 = fig.add_subplot(gs[0, 0])
           
            plot_peak_dist(pick_locs,
                           hist_data,
                           estimated_peaks,
                           r_par,
                           binning=binning,
                           cutoff=cutoff,
                           axes="y",
                           ax=ax1)
            
            peak_dist = abs(ring_parameter[0][1]-ring_parameter[1][1])
        
            ################
            # (2) isolate the locs from every ring
            # fitted center +- peak-to-peak distance/2
            ################
            
            
            for id, ring in enumerate(ring_parameter):
                
                print("ring {} with limits {} & {}".format(id,ring[1]-(ring[2]*cutoff),ring[1]+(ring[2]*cutoff)))
                # isolate the locs from every ring
                r_locs = isolate_ring(pick_locs, c=ring[1], w=ring[2], cutoff=cutoff, axes="y")
                ring_locs[id] = r_locs
                
                # link the isolated locs
                r_locs_linked = link(r_locs, info, max_dist, max_dark_time)
                ring_locs_linked[id] = r_locs_linked
                
                # extract number of locs
                ring_n_events[id] = len(r_locs_linked)
              
                        
            ################
            # (3b) calculate ring radius by rotation into plane of the ring and circle fitting in 2D projection
            ################
            
            ax3 = fig.add_subplot(gs[1, 0])
            ax5 = fig.add_subplot(gs[2, 0])
            normal = {}
            for id, rings in ring_locs.items():
                
                # prepare XYZ coordinates
                XYZ = np.array([rings.x, rings.y, rings.z]).transpose()

                # rotate coordinates into the plane of the ring. ring should be now in X-Y
                ring_rot[id], normal[id] = rotate_ring(XYZ)
                
                # fit 2D ring into the XY ring data
                x_center, y_center, Radius, residu = leastsq_circle(ring_rot[id][:,0], ring_rot[id][:,1])
                
                c = rodrigues_rot(np.array([x_center,y_center,0]), [0,0,1], normal[id]) + XYZ.mean(axis=0)
                center[id] = c.flatten()
                
                # extract radii
                ring_radii[id] = Radius
                
                # plot
                if id == 0:
                    ax_circ = ax3
                if id == 1:
                    ax_circ = ax5
                
                ringing = pd.DataFrame([ring_rot[id][:,0]-x_center,
                                        ring_rot[id][:,1]-y_center]).transpose()
                print("pick {} radius {}".format(id, Radius))
                ringing.to_csv("ring_file_{}_pick_{}".format(pick, id))
                plot_data_circle(ring_rot[id][:,0],
                                 ring_rot[id][:,1],
                                 x_center,
                                 y_center,
                                 Radius,
                                 id,
                                 center=True,
                                 ax=ax_circ)
            
            # calculate the center-to-center distance
            center_dist = distance.euclidean(center[0],center[1])
                
            # calculate the angle between to two fitted ring planes
            ring_angle = abs(angle_between(normal[0],normal[1]))
            
            # if normals point into opposite direction
            if ring_angle > 90:
                ring_angle -=180
                ring_angle = abs(ring_angle)
               
            # calculate the angle between coverslip and spore or mother ring
            coverslip_s_angle = angle_between(normal[0],[0,0,1])
            coverslip_m_angle = angle_between(normal[1],[0,0,1])
            
            # plot 3D ring
            ax2 = fig.add_subplot(gs[0, 1], projection="3d")
            plot_3d_ring(ring_rot,center_dist, ax=ax2)
            
            ################
            # (4) qPAINT analysis
            ################
            
            ax4 = fig.add_subplot(gs[1, 1])
            ax6 = fig.add_subplot(gs[2, 1])
            
            for id, rings_linked in ring_locs_linked.items():

                    # compute kinetics
                    kinetics = compute_dark_times(rings_linked)

                    # compute mean bright and dark times
                    ring_kinetics[id] = {"bright":np.nanmean(kinetics.len),
                                         "dark":np.nanmean(kinetics.dark)}
                    
                    fit_result_len = fit_cum_exp(kinetics.len)
                    fit_result_dark = fit_cum_exp(kinetics.dark)
                
                    if id == 0:
                        ax_fit=ax4
                    if id == 1:
                        ax_fit=ax6
                    
                    plot_cum_exp(kinetics, fit_result_len, fit_result_dark, id, ax=ax_fit)
                    
                    ring_kinetics_fit[id] = {"bright":np.nanmean(fit_result_len.best_values["t"]),
                                             "dark":np.nanmean(fit_result_dark.best_values["t"])}
                       
            ################
            # (5) append data into large array
            ################
            
            ring_data.append([file_id, #running file index
                              filename, #filename
                              pick, #pick number
                              ring_radii[0], #fitted radius of spore (nm)
                              ring_radii[1], #fitted radius of mother (nm)
                              peak_dist, #peak-to-peak distance (nm)
                              center_dist, #center-to-center distance (nm)
                              ring_angle, #angle between the two ring planes (°)
                              coverslip_s_angle, #anle between spore ring and coverslip (°)
                              coverslip_m_angle, #anle between mother ring and coverslip (°)
                              ring_n_events[0], #number of locs in spore ring
                              ring_n_events[1], #number of  locs in mother ring
                              ring_kinetics[0]["bright"], #mean bright time of spore ring (frames)
                              ring_kinetics[0]["dark"], #mean dark time of spore ring (frames)
                              ring_kinetics[1]["bright"], #mean bright time of mother ring (frames)
                              ring_kinetics[1]["dark"], #mean dark time of mother ring (frames)
                              ring_kinetics_fit[0]["bright"], #cumulative mean bright times of spore (frames)
                              ring_kinetics_fit[0]["dark"], #cumulative mean dark times of spore (frames)
                              ring_kinetics_fit[1]["bright"], #cumulative mean bright times of mother (frames)
                              ring_kinetics_fit[1]["dark"] #cumulative mean dark times of mother (frames)
                             ])
            
            ################
            # (6) save plots
            ################
            
            img_fname = "file_{}_pick_{}".format(file_id, pick)
            img_name = os.path.join(analysis_folder, img_fname)
            plt.savefig(img_name+img_format, dpi=dpi, format="svg")
            plt.close(fig)
            
            
            ################
            # (7) export ring locs
            ################
            
            for id, rings_linked in ring_locs_linked.items():
                save_ring_locs(rings_linked, info, analysis_folder, file_id, pick, id, link=True)
            
            for id, rings in ring_locs.items():
                save_ring_locs(rings, info, analysis_folder, file_id, pick, id, link=False)
            
        else:
            fig, ax = plt.subplots(1, 1, figsize=(7, 5), constrained_layout=True)
            fig.suptitle(("Pick {} - Ring analysis\n"
                          "File: {}").format(pick,filename), fontsize=14, ha="center")
            
            plot_peak_dist(pick_locs, hist_data, estimated_peaks, r_par, binning=binning, axes="y", ax=ax)
            img_fname = "file_{}_pick_{}".format(file_id, pick)
            img_name = os.path.join(analysis_folder_excluded, img_fname)
            plt.savefig(img_name+img_format, format="svg")
            plt.close(fig)
            rings_excluded.append(pick)
        
print("Calculation finished.")
print("Total: {} rings analysed, {} excluded.".format(len(ring_data), len(rings_excluded)))

File 1 Pick 0
        0           1
0     6.0  141.548218
1    11.0  155.841492
2     7.0  170.134766
3    20.0  184.428040
4    11.0  198.721313
5     5.0  213.014587
6     6.0  227.307861
7     7.0  241.601135
8     5.0  255.894409
9     5.0  270.187683
10   12.0  284.480957
11   61.0  298.774231
12    8.0  313.067505
13    6.0  327.360779
14   34.0  341.654053
15   18.0  355.947327
16   32.0  370.240601
17   27.0  384.533875
18   44.0  398.827148
19   84.0  413.120422
20  122.0  427.413696
21  132.0  441.706970
22  180.0  456.000244
23  215.0  470.293518
24  159.0  484.586792
25  117.0  498.880066
26  105.0  513.173340
27  130.0  527.466614
28  116.0  541.759888
29  168.0  556.053162
30  223.0  570.346436
31  187.0  584.639709
32  110.0  598.932983
33   81.0  613.226257
34   76.0  627.519531
35   76.0  641.812805
36   30.0  656.106079
37    7.0  670.399353
38    6.0  684.692627
39    5.0  698.985901
40    2.0  713.279175
41    1.0  727.572449
42    3.0  741.865723
43    2.0  756.158

pick 0 radius 287.54778620708805
pick 1 radius 325.2180907082093
File 1 Pick 7
        0           1
0     7.0   33.290127
1     2.0   48.631676
2     1.0   63.973221
3     3.0   79.314766
4    12.0   94.656319
5     7.0  109.997864
6     2.0  125.339409
7     1.0  140.680954
8     6.0  156.022507
9     6.0  171.364044
10    7.0  186.705597
11   10.0  202.047150
12   21.0  217.388687
13    8.0  232.730240
14   17.0  248.071793
15   18.0  263.413330
16   33.0  278.754883
17  112.0  294.096436
18  130.0  309.437988
19   71.0  324.779510
20   27.0  340.121063
21   18.0  355.462616
22   34.0  370.804169
23   84.0  386.145721
24  108.0  401.487274
25   75.0  416.828796
26   77.0  432.170349
27   58.0  447.511902
28   32.0  462.853455
29   48.0  478.195007
30   47.0  493.536530
31   30.0  508.878082
32   22.0  524.219666
33   14.0  539.561157
34    9.0  554.902710
35    2.0  570.244263
36    0.0  585.585815
37    0.0  600.927368
38    1.0  616.268921
39    1.0  631.610474
40    3.0  646.9520

File 1 Pick 14
        0           1
0     2.0   25.504963
1     0.0   43.728920
2     1.0   61.952877
3     1.0   80.176834
4     0.0   98.400787
5     1.0  116.624748
6     0.0  134.848694
7     0.0  153.072662
8     4.0  171.296616
9     1.0  189.520569
10    0.0  207.744522
11    0.0  225.968475
12    1.0  244.192444
13    5.0  262.416382
14    3.0  280.640350
15    3.0  298.864319
16    0.0  317.088257
17    3.0  335.312225
18    5.0  353.536163
19    5.0  371.760132
20    9.0  389.984100
21    5.0  408.208038
22   21.0  426.432007
23   33.0  444.655945
24   28.0  462.879913
25   48.0  481.103882
26   52.0  499.327820
27   43.0  517.551758
28   53.0  535.775757
29  112.0  553.999695
30  121.0  572.223633
31  137.0  590.447632
32   80.0  608.671570
33  142.0  626.895508
34  120.0  645.119507
35  111.0  663.343445
36   72.0  681.567383
37   56.0  699.791321
38   36.0  718.015320
39   24.0  736.239258
40   16.0  754.463196
41   14.0  772.687195
42   13.0  790.911133
43    4.0  809.13

## Save analysis data

In [ ]:
df_ring_data = pd.DataFrame(ring_data, columns=["file_index",
                                                "filename",
                                                "group",
                                                "radius_spore",
                                                "radius_mother",
                                                "p2p distance",
                                                "c2c distance",
                                                "angle_between_rings",
                                                "angle_between_s_ring_and_coverslip",
                                                "angle_between_m_ring_and_coverslip",
                                                "n_events_spore",
                                                "n_events_mother",
                                                "mean_bright_spore",
                                                "mean_dark_spore",
                                                "mean_bright_mother",
                                                "mean_dark_mother",
                                                "fit_bright_spore",
                                                "fit_dark_spore",
                                                "fit_bright_mother",
                                                "fit_dark_mother"
                                               ])
df_ring_data.to_csv("ring_data.csv")
print("Data saved to CSV file in locs folder.")